# Import a Molecules/Atom from cartesian coordinates:
**Task:** 
1. Create a Molecule object from its cartesian coordinates
2. Ask for its atomic and molecular properties

## [Water molecule](https://en.wikipedia.org/wiki/Properties_of_water) ($H_{2}O$):

<img 
amcess="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/H2O_2D_labelled.svg/2560px-H2O_2D_labelled.svg.png"
alt="water" 
width="200" 
height="100" 
/>

### Properties:
- Number of molecules: 1
- Number of atoms: 3
- Atomic symbols: O, H, H
- Atomic masses [g/mol]: 
    - O: 15.999
    - H: 1.008
    - H: 1.008
- Molar mass [g/mol]: 18.015
- Cartesian coordinates [Angstrom]:
    ```XYZ
    O    0.00000    0.00000    0.00000
    H    0.58708    0.75754    0.00000
    H    0.58708   -0.75754    0.00000
    ```
- Bond distance [Angstrom]:
    - H-O: 0.9584 (95.84 pm)
    - H-H: 1.5151
- Bond angle [Degrees]:
    - H-O-H: 104.45
    - H-H-O: 37.78

In [1]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'amcess' directory is out ('..') of our current directory 

import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)

/home/danian/udemy/amcess


In [2]:
# Import Molecule Class
from amcess.base_molecule import Molecule, Cluster

In [3]:
from copy import deepcopy
import numpy as np


def controled_move(initial_cluster: Cluster, displacements,angular_rotations):
    final_cluster = deepcopy(initial_cluster)
    for i in range(final_cluster.total_molecules-1):
        final_cluster = final_cluster.translate(i, displacements[i][0],
                                                    displacements[i][1],
                                                    displacements[i][2])
        final_cluster = final_cluster.rotate(i,angular_rotations[i][0],
                                               angular_rotations[i][1],
                                               angular_rotations[i][2])                                            
                                                    
    return final_cluster


#print(tri_water.translate(0, 1,1,1).rotate(0,10,10,10).translate(1, 1,1,1).rotate(1,10,10,10).xyz)
#displacements = [(1,1,1)]*2
#angular_rotations = [(10,10,10)]*2
#final = controled_move(tri_water, displacements,angular_rotations)
#print(final.xyz)

In [5]:
def visualization_random_process(serie_string : str, n_atoms):
    #Approximately 3 bohr radius for each atom
    sphere_radius = n_atoms*0.5*0.5
    xyz_view = py3Dmol.view(width=1000,height=700)
    xyz_view.addModelsAsFrames(serie_string,'xyz')
    xyz_view.setStyle({'sphere': {'radius': 0.8}})
    xyz_view.addSphere({'center': {'x':0, 'y':0, 'z':0}, 
        'radius': sphere_radius , 
        'color' :'yellow',
        'alpha': 0.5,
        })

    xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 10})
    xyz_view.zoomTo()
    xyz_view.show()
#visualization_random_process(four_water.xyz, 12)


In [6]:
from scipy.optimize import shgo
def optimization_shgo(cluster_of_interest):

    sphere_radius = cluster_of_interest.total_atoms*0.5*0.5
    discretization = sphere_radius / 1.6
    bounds = np.array([(-discretization,discretization),
                        (-discretization,discretization),
                        (-discretization,discretization),
                        (0,360),(0,360),(0,360)])
    shgo_tolerance_dict = {"ftol" : 1e-6} # Default is 1e-12
    shgo_options_dict = {"options" : shgo_tolerance_dict}
    bounds_full = np.repeat(bounds,cluster_of_interest.total_molecules-1,axis=0)
    print("bounds full ",len(bounds_full),cluster_of_interest.total_molecules,bounds_full)

    minimos = shgo(optimization_kernel, bounds=bounds_full, minimizer_kwargs=shgo_options_dict, sampling_method='sobol',n=32, iters=1)

    return minimos



from pyscf import gto, scf
def optimization_kernel(x):
#def optimization_juan_inner(x,*kwargs): #to test if I can send some objects here
    global serie_of_interest_xyz_last
    #No tengo modos de meter como argumento la molécula... 
    displacements= [ np.zeros(3) ]
    angular_rotations= [ np.zeros(3) ]
    for i in range(cluster_of_interest.total_molecules-1):
        displacements.append(x[i*3:i*3+3])
        angular_rotations.append(x[(cluster_of_interest.total_molecules-1+i)*3:(cluster_of_interest.total_molecules+i)*3])
        
    print("r ",displacements)
    print("theta ",angular_rotations)
    candidate_to_test = controled_move(cluster_of_interest, displacements, angular_rotations)
    serie_of_interest_xyz_last += candidate_to_test.xyz
    mol = gto.Mole()
    mol.fromstring(str(serie_of_interest_xyz_last))
    mol.build(basis = 'sto-3g')
    #mol.build(basis = '6-31**')
    from pyscf import dft
    if (RHF):
        mf = scf.RHF(mol)
    if (DFT):

        mf = dft.RKS(mol)
        mf.xc = 'b3lyp,b3lyp'
    mf.kernel()
    return mf.energy_tot()
#    return candidate_to_test   
#cluster_of_interest = tri_water
#x = np.array([0,0,0,2,2,2,100,100,100,100,100,100])
#y = np.array([2,2,2,100,100,100])
#cand = optimization_kernel(x)
#print("la energía es",cand)


In [7]:
def optimization_dual_annealing(cluster_of_interest):
    from scipy.optimize import dual_annealing

    sphere_radius = cluster_of_interest.total_atoms*0.5*0.5
    discretization = sphere_radius / 1.6
    bounds = np.array([(-discretization,discretization),(-discretization,discretization),(-discretization,discretization),(0,360),(0,360),(0,360)])

    bounds_full = np.repeat(bounds,cluster_of_interest.total_molecules-1,axis=0)
    bounds_full=np.array(bounds_full)

    lista_moves=[]
    lista_values=[]
    for iteracion in range (0,5):
        minimos = dual_annealing(optimization_kernel,bounds=bounds_full, maxiter=1, initial_temp=5230, local_search_options={"method" : 'BFGS'},no_local_search=True , restart_temp_ratio = 0.9)

        lista_moves.append(minimos.x)
        lista_values.append(minimos.fun)


    return lista_values, lista_moves

In [8]:
# AMCESS test
water=[("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]
water_molecule = Cluster(water)


import py3Dmol

#water_molecule = Molecule(water_dict)
#water_molecule = water_molecule.translate(0, x=1.5, y=1.5, z=1.5)
bi_water = water_molecule*2
#tri_water = water_molecule*3
#four_water = water_molecule*4

bi_water = bi_water.initialize_cluster(max_closeness=3)
#tri_water = tri_water.initialize_cluster(max_closeness=3)
#four_water = four_water.initialize_cluster(max_closeness=3)

print(bi_water.xyz)



cluster_of_interest = bi_water
shgo_bool = False
dual_annealing_bool = True

RHF = False
DFT = True

#Choose wheter doing the re-optimization in two steps, or not.
first_constraining = True




serie_of_interest_xyz_last = "" # Variable global...

if (shgo_bool):
    minimos = optimization_shgo(cluster_of_interest)
    print(minimos)

if (dual_annealing_bool):
    minimos , movimientos = optimization_dual_annealing(cluster_of_interest)



	6
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	    -2.68978607	    -1.40833169	     2.43678185
H     	    -2.59142501	    -0.51194651	     2.11220752
H     	    -3.48102942	    -0.95760167	     2.73562327



TypeError: dual_annealing() got an unexpected keyword argument 'local_search_options'

In [78]:
#Ploteo todos los puntos donde se evaluó la energía
minimos
#rint(serie_of_interest_xyz_last)
#visualization_random_process(serie_of_interest_xyz_last,cluster_of_interest.total_atoms)

[-169.5018312457233,
 -169.50183124572325,
 -169.50183124572337,
 -169.5018312457233,
 -169.5018312457233]

In [81]:
candidates=[]
candidates_coord=[]

if (shgo_bool):

        for step in range(len(minimos.xl)):
                desplazamientos = [ np.zeros(3) ]
                rotaciones = [ np.zeros(3) ]
                for i in range(cluster_of_interest.total_molecules-1):
                        desplazamientos.append(minimos.xl[step][i*3:i*3+3])
                        rotaciones.append(minimos.xl[step][(cluster_of_interest.total_molecules-1+i)*3:(cluster_of_interest.total_molecules+i)*3])


        for i in range(len(minimos.xl)):
                retrieved_candidated = controled_move(cluster_of_interest, desplazamientos, rotaciones)
                candidates.append(retrieved_candidated)
                candidates_coord.append(retrieved_candidated.coordinates)
        if len(minimos.xl) > 1:
                #Elimino candidatos que sean iguales (coordenadas todas muy parecidas)
                for i in range(len(minimos.xl)-1):
                        for k in range(i+1,len(candidates)):
                                if np.all(np.isclose(candidates[i].coordinates, candidates[k].coordinates,atol=0.05)):
                                        candidates = np.delete(candidates,[k]) #valores muy cercanos
                                else:
                                        pass


if (dual_annealing_bool):
        for step in range(len(movimientos)):
                desplazamientos = [ np.zeros(3) ]
                rotaciones = [ np.zeros(3) ]
                for i in range(cluster_of_interest.total_molecules-1):
                        desplazamientos.append(movimientos[step][i*3:i*3+3])
                        rotaciones.append(movimientos[step][(cluster_of_interest.total_molecules-1+i)*3:(cluster_of_interest.total_molecules+i)*3])


                retrieved_candidated = controled_move(cluster_of_interest, desplazamientos, rotaciones)
                candidates.append(retrieved_candidated)
                candidates_coord.append(retrieved_candidated.coordinates)
        
candidates

[Cluster(_atoms=[('O', 0.0, 0.0, 0.0), ('H', 0.58708, 0.7575399999999999, 0.0), ('H', -0.58708, 0.7575399999999999, 0.0), ('O', 1.7674706358772414, -1.3558320692480326, -2.1222212225177834), ('H', 2.6332520374237256, -0.9488374878818027, -2.179734294725177), ('H', 1.6129423101713036, -0.528811954714272, -2.581229397511525)], _charge=0, _multiplicity=1),
 Cluster(_atoms=[('O', 0.0, 0.0, 0.0), ('H', 0.58708, 0.7575399999999999, 0.0), ('H', -0.58708, 0.7575399999999999, 0.0), ('O', 1.7674706358772414, -1.3558320692480326, -2.1222212225177834), ('H', 2.6332520374237256, -0.9488374878818027, -2.179734294725177), ('H', 1.6129423101713036, -0.528811954714272, -2.581229397511525)], _charge=0, _multiplicity=1),
 Cluster(_atoms=[('O', 0.0, 0.0, 0.0), ('H', 0.58708, 0.7575399999999999, 0.0), ('H', -0.58708, 0.7575399999999999, 0.0), ('O', 1.7674706358772414, -1.3558320692480326, -2.1222212225177834), ('H', 2.6332520374237256, -0.9488374878818027, -2.179734294725177), ('H', 1.6129423101713036, -0.

In [83]:
from pyscf import dft
#Save results on file

#Clean file
with open('water.xyz', 'w') as f:
                f.write(" ")


for i in range(len(candidates)):
        mol = gto.Mole()
        mol.fromstring(str(candidates[i].xyz))
        mol.build()

        if (RHF):
                mf = scf.RHF(mol)
        if (DFT):
                mf = dft.RKS(mol)
                mf.xc = 'b3lyp,b3lyp'
        mf.kernel()
        energia = mf.energy_tot()


        with open('water.xyz', 'a') as f:

                f.write("---------------    Candidate %.0f    ---------------\n" % int(i+1))
                f.write(str(candidates[i]))
                f.write("Energy: %.6f \n" % mf.energy_tot())


# Save only molecule (not energy)
np.save("water-candidates.npy", candidates)

converged SCF energy = -169.501831245723
converged SCF energy = -169.501831245723
converged SCF energy = -169.501831245723
converged SCF energy = -169.501831245723
converged SCF energy = -169.501831245723


In [91]:
## See Final molecule

import py3Dmol

#for i in range(len(candidates)):
#    xyz_view = py3Dmol.view(width=300,height=200)
#    xyz_view.addModel(str(candidates[i]),'xyz')
#    xyz_view.setStyle({'stick': {'radius': .15}, 'sphere': {'scale': 0.3}})

#    xyz_view.zoomTo()
#    xyz_view.show()


A=np.load("water-candidates.npy", allow_pickle=True)


for i in range(len(A)):
    xyz_view = py3Dmol.view(width=300,height=200)
    xyz_view.addModel(str(A[i].xyz),'xyz')
    xyz_view.setStyle({'stick': {'radius': .15}, 'sphere': {'scale': 0.3}})

    xyz_view.zoomTo()
    xyz_view.show()



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [90]:
#Creating files for imposing constraints to the reoptimizations
#Constraints format on https://github.com/leeping/geomeTRIC/blob/master/examples/constraints.txt

#By default, performs a geometry optimization with constraints and then
#another one with no constraints (beggining from the first one). 

from pyscf.geomopt.geometric_solver import optimize
from pyscf.tools import molden



#Átomos por fragmento (debe crearse un nuevo atributo en la clase Molecule)
porfrag=3

for i in range(len(candidates)):
    with open("constraints_%s.txt" % i, 'w') as f:
        f.write("$freeze\n")


    bond_var = 1
    B=candidates[i]
    for j in range(B.total_molecules):
        for k in range(1,porfrag):
            for l in range(k+1,porfrag+1):
                #There are N(N-1)/2 lenghts for each fragment (N is the number of atoms of each fragment)
                with open("constraints_%s.txt" % i, 'a') as f:
                        f.write("distance {} {}\n".format(bond_var-1+k,bond_var-1+l))
        bond_var = bond_var + porfrag



#Reoptimize candidates

A=np.load("water-candidates.npy", allow_pickle=True)

#Clean files
with open('energy_hartrees.txt', 'w') as f:
    f.write("Molecule   Energy\n")
with open('energy_kcalmol.txt', 'w') as f:
    f.write("Molecule   Energy\n")


for i in range(len(A)):

    mol = gto.Mole()
    mol.fromstring(str(A[i].xyz))
    mol.build(basis = 'sto-3g')

    if (RHF):
        mf = scf.RHF(mol)
    if (DFT):
        mf = dft.RKS(mol)
        mf.xc = 'b3lyp,b3lyp'
    mf.kernel()



    if (first_constraining):
        #Reoptimization with constraints
        params = {"constraints": "constraints_%s.txt" % i,}
        mol_eq = mf.Gradients().optimizer(solver='geomeTRIC').kernel(params)
        #Export optimized molecule to be opened with Jmol (not used now)
        #molden.from_mo(mol_eq, "molecula_%.0f_constraint.molden" % i, mf.mo_coeff)




    #Reoptimization without constraints
    if not (first_constraining):
        pass
    else:
        mf = scf.RHF(mol_eq)
        mf.kernel()

    mol_eq = optimize(mf, maxsteps=100)
    #Export optimized molecule to be opened with Jmol
    molden.from_mo(mol_eq, "molecula_%.0f.molden" % i, mf.mo_coeff)

    energy_hartrees = mf.energy_tot()
    energy_kcalmol = energy_hartrees * 627.5096080305927

    with open('energy_hartrees.txt', 'a') as f:
        f.write("\t{}\t\t{:.2f}\n".format(i,energy_hartrees))
    with open('energy_kcalmol.txt', 'a') as f:
        f.write("\t{}\t\t{:.2f}\n".format(i,energy_kcalmol))


converged SCF energy = -169.501831245723



                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              ********      .))))))  
          ##      .%%%%%%/      (%%%%%%,                  ,******      /)))))  
        %%%%%%      .%%%%%%#      *%%%%%%,    ,/////.       ******      )))))) 
      #%      %%      .%%%%%%/      *%%


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   0.587080   0.757540   0.000000    0.000000  0.000000  0.000000
   H  -0.587080   0.757540   0.000000    0.000000  0.000000  0.000000
   O   1.767471  -1.355832  -2.122221    0.000000  0.000000  0.000000
   H   2.633252  -0.948837  -2.179734    0.000000  0.000000  0.000000
   H   1.612942  -0.528812  -2.581229    0.000000  0.000000  0.000000
converged SCF energy = -169.50183124567
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 O     0.0010410790    -0.2798189405    -0.0006364663
1 H    -0.0024588837     0.1391309584    -0.0010982541
2 H     0.0037206210     0.1379847429    -0.0007318815
3 O    -0.1338449780    -0.2238731510     0.0986988331
4 H     0.0624040966     0.1139211114    -0.0484638351
5 H     0.0691394208     0.1126396490 

Step    0 : Gradient = 2.644e-03/5.453e-03 (rms/max) Energy = -169.5018312457
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 4.76100e-01 5.58191e-01 5.58191e-01



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.037529   0.028786   0.023959   -0.037529  0.028786  0.023959
   H   0.569075   0.770751   0.030887   -0.018005  0.013211  0.030887
   H  -0.604662   0.801372   0.023471   -0.017582  0.043832  0.023471
   O   1.831633  -1.414747  -2.175627    0.064162 -0.058915 -0.053406
   H   2.657797  -0.930550  -2.214813    0.024545  0.018287 -0.035079
   H   1.597308  -0.573518  -2.570540   -0.015635 -0.044706  0.010689

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess density matrix may have large error.

converged SCF energy = -169.503413053864
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 O    -0.0068651262    -0.2794238344    -0.0005249879
1 H     0.0011816839     0.1388492206    -0.0009972181
2 H     0.0074229024     0.1379380092    -0.0002937427
3 O    -

...

Step   11 : Displace = 2.664e-03/4.114e-03 (rms/max) Trust = 3.000e-01 (=) Grad = 1.221e-04/1.624e-04 (rms/max) E (change) = -149.9412433502 (-2.003e-06) Quality = 1.433
Hessian Eigenvalues: 6.89145e-03 3.63790e-02 4.41196e-02 ... 5.72227e-01 6.10868e-01 6.74997e-01



Geometry optimization cycle 13
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.258291   0.437799  -0.628963    0.002629 -0.000914  0.001412
   H   0.687645   1.266118  -0.306699   -0.001641  0.001382  0.001070
   H  -0.023964   0.029124   0.224049   -0.001602  0.001415  0.001080
   O   2.070713  -1.159207  -1.921075    0.000126 -0.000537 -0.001064
   H   1.414849  -0.582353  -1.456007   -0.000048 -0.000564 -0.001351
   H   1.619965  -1.274356  -2.792789    0.000536 -0.000782 -0.001147
converged SCF energy = -149.941244261233
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 O    -0.0000082437     0.0000008116    -0.0000094208
1 H     0.0000203944     0.0000210570    -0.0000162520
2 H    -0.0000100072    -0.0000314854     0.0000061714
3 O     0.0000303569    -0.0000076616     0.0000221339
4 H    -0.0000021088     0.0000048845     0.0000086626
5 H    -0.0000303917     0.000012393

Step   12 : Displace = 2.109e-03/2.971e-03 (rms/max) Trust = 3.000e-01 (=) Grad = 2.941e-05/3.834e-05 (rms/max) E (change) = -149.9412442612 (-9.110e-07) Quality = 1.144
Hessian Eigenvalues: 7.28604e-03 3.05065e-02 4.25567e-02 ... 5.72181e-01 6.11563e-01 6.67604e-01



Geometry optimization cycle 14
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.258321   0.437931  -0.628609    0.000030  0.000132  0.000354
   H   0.687493   1.266316  -0.306315   -0.000152  0.000198  0.000384
   H  -0.024067   0.029395   0.224410   -0.000103  0.000272  0.000361
   O   2.070687  -1.159198  -1.921081   -0.000026  0.000009 -0.000006
   H   1.414289  -0.582359  -1.456764   -0.000560 -0.000006 -0.000757
   H   1.620776  -1.274961  -2.793126    0.000810 -0.000605 -0.000336
converged SCF energy = -149.941244312114
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 O     0.0000036400    -0.0000021870    -0.0000002995
1 H     0.0000019683     0.0000039665    -0.0000031396
2 H    -0.0000031024    -0.0000045568     0.0000003897
3 O     0.0000037442     0.0000009120     0.0000065465
4 H    -0.0000005096    -0.0000009151    -0.0000027879
5 H    -0.0000057404     0.000002780

Step   13 : Displace = 6.570e-04/1.052e-03 (rms/max) Trust = 3.000e-01 (=) Grad = 5.566e-06/7.597e-06 (rms/max) E (change) = -149.9412443121 (-5.088e-08) Quality = 1.107
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    
Time elapsed since start of run_optimizer: 7.150 seconds


In [ ]:
# Atomic Simulation Environment
# https://wiki.fysik.dtu.dk/ase/index.html
# !pip install --upgrade --user ase


# ChemML
# https://hachmannlab.github.io/chemml/index.html
# !pip install chemml

In [ ]:
# NGLview
# https://github.com/nglviewer/nglview
# !pip install nglview

# ---------------------------------------
# pytraj 
# https://amber-md.github.io/pytraj/latest/index.html
# !pip install pytraj

